In [ ]:
!pip install -q torch torchvision pycocotools opencv-python

In [ ]:
import zipfile

zip_name = "/content/My First Project.v1i.coco.zip"

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall("dataset")


In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
import torchvision.ops as ops

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict

# === Dataset Transform ===
class CocoTransform:
    def __call__(self, image, target):
        return F.to_tensor(image), target

# === Dataset Yükleyici ===
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )

train_dataset = get_coco_dataset("/content/dataset/train", "/content/dataset/train/_annotations.coco.json")
val_dataset = get_coco_dataset("/content/dataset/valid", "/content/dataset/valid/_annotations.coco.json")

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# === Model Tanımı ===
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

num_classes = 6  # background + 5 sınıf
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model(num_classes)
model.to(device)

# === Optimizer ve Scheduler ===
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# === Log listeleri ===
train_losses = []
val_precisions = []
val_recalls = []
val_map50 = []
val_map5095 = []

# === Eğitim Fonksiyonu ===
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0

    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        processed_targets = []
        valid_images = []

        for i, target in enumerate(targets):
            boxes = []
            labels = []
            for obj in target:
                x, y, w, h = obj["bbox"]
                if w > 0 and h > 0:
                    boxes.append([x, y, x + w, y + h])
                    labels.append(obj["category_id"])
            if boxes:
                processed_target = {
                    "boxes": torch.tensor(boxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device),
                }
                processed_targets.append(processed_target)
                valid_images.append(images[i])

        if not processed_targets:
            continue

        images = valid_images
        loss_dict = model(images, processed_targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    avg_loss = total_loss / len(data_loader)
    train_losses.append(avg_loss)
    print(f"[Epoch {epoch}] Avg Train Loss: {avg_loss:.4f}")

# === Değerlendirme Fonksiyonu ===
def evaluate_model(model, data_loader, device, iou_threshold=0.5, score_threshold=0.5):
    model.eval()
    total_tp = defaultdict(int)
    total_fp = defaultdict(int)
    total_fn = defaultdict(int)
    ap_per_class = defaultdict(list)

    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for output, target in zip(outputs, targets):
                if len(target) == 0:
                    continue

                pred_boxes = output['boxes']
                pred_labels = output['labels']
                pred_scores = output['scores']

                gt_boxes = torch.tensor([obj["bbox"] for obj in target], dtype=torch.float32).to(device)
                if gt_boxes.ndim == 1:
                    gt_boxes = gt_boxes.unsqueeze(0)
                gt_boxes[:, 2] += gt_boxes[:, 0]
                gt_boxes[:, 3] += gt_boxes[:, 1]
                gt_labels = torch.tensor([obj["category_id"] for obj in target]).to(device)

                keep = pred_scores > score_threshold
                pred_boxes = pred_boxes[keep]
                pred_labels = pred_labels[keep]

                matched_gt = set()
                for i in range(len(pred_boxes)):
                    box = pred_boxes[i].unsqueeze(0)
                    label = pred_labels[i].item()
                    ious = ops.box_iou(box, gt_boxes)
                    iou_max, idx = ious.max(1)
                    idx = idx.item()

                    if iou_max.item() >= iou_threshold and gt_labels[idx].item() == label and idx not in matched_gt:
                        total_tp[label] += 1
                        matched_gt.add(idx)
                        ap_per_class[label].append(iou_max.item())
                    else:
                        total_fp[label] += 1

                for i, lbl in enumerate(gt_labels):
                    if i not in matched_gt:
                        total_fn[lbl.item()] += 1

    total_p, total_r, total_ap50, total_ap5095, count = 0, 0, 0, 0, 0

    for label in total_tp:
        tp = total_tp[label]
        fp = total_fp[label]
        fn = total_fn[label]
        p = tp / (tp + fp) if (tp + fp) > 0 else 0
        r = tp / (tp + fn) if (tp + fn) > 0 else 0
        aps = ap_per_class[label]
        ap50 = sum([1 if ap > 0.5 else 0 for ap in aps]) / len(aps) if aps else 0
        ap5095 = sum(aps) / len(aps) if aps else 0
        total_p += p
        total_r += r
        total_ap50 += ap50
        total_ap5095 += ap5095
        count += 1

    avg_p = total_p / count
    avg_r = total_r / count
    avg_ap50 = total_ap50 / count
    avg_ap5095 = total_ap5095 / count

    val_precisions.append(avg_p)
    val_recalls.append(avg_r)
    val_map50.append(avg_ap50)
    val_map5095.append(avg_ap5095)

    print(f"Precision: {avg_p:.3f}, Recall: {avg_r:.3f}, mAP50: {avg_ap50:.3f}, mAP50-95: {avg_ap5095:.3f}")

# === Eğitim Döngüsü ===
num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()
    evaluate_model(model, val_loader, device)
    torch.save(model.state_dict(), f"fasterrcnn_epoch_{epoch+1}.pth")

# === Grafik Çizimi ===
metrics = [
    train_losses, val_precisions, val_recalls,
    val_map50, val_map5095
]

titles = [
    "train/loss", "val/precision", "val/recall",
    "metrics/mAP50", "metrics/mAP50-95"
]

plt.figure(figsize=(15, 8))
for i, data in enumerate(metrics):
    plt.subplot(2, 3, i + 1)
    plt.plot(data, label='value', marker='o')
    if len(data) >= 5:
        plt.plot(pd.Series(data).rolling(3).mean(), linestyle='dotted', label='smooth')
    plt.title(titles[i])
    plt.legend()
    plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# Dataset class
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )

# Load datasets
train_dataset = get_coco_dataset(
    img_dir="/content/dataset/train",
    ann_file="/content/dataset/train/_annotations.coco.json"
)


val_dataset = get_coco_dataset(
    img_dir="/content/dataset/valid",
    ann_file="/content/dataset/valid/_annotations.coco.json"
)



# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
# Load Faster R-CNN with ResNet-50 backbone
def get_model(num_classes):
    # Load pre-trained Faster R-CNN
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# Initialize the model
num_classes = 6 # Background + car, bus, truck, motorcycle, ambulance
model = get_model(num_classes)

In [ ]:
# @title
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from PIL import Image

# Sınıf isimleri
COCO_CLASSES = {
    0: "Background",
    1: "ambulance",
    2: "bus",
    3: "car",
    4: "motorcycle",
    5: "truck"
}

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

num_classes = 6
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Modeli yükle
model = get_model(num_classes)
model.load_state_dict(torch.load("/content/fasterrcnn_epoch_10.pth"))
model.to(device)
model.eval()

# Görseli hazırla
def prepare_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = F.to_tensor(image).unsqueeze(0)
    return image_tensor.to(device)

# Sınıf adını getir
def get_class_name(class_id):
    return COCO_CLASSES.get(class_id, "Unknown")

# Kutuları çiz
def draw_boxes(image, prediction, fig_size=(12, 10), threshold=0.5):
    boxes = prediction[0]['boxes'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()

    plt.figure(figsize=fig_size)
    plt.imshow(image)

    ax = plt.gca()

    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            x_min, y_min, x_max, y_max = box
            class_name = get_class_name(label)
            rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                 linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x_min, y_min - 5, f"{class_name} ({score:.2f})", color='red', fontsize=12,
                    backgroundcolor="white")

    plt.axis('off')
    plt.show()

# Tahmin yap ve çiz
image_path = "/content/dataset/valid/f971dfb20ee52e9b_jpg.rf.a6484c46c580abcacfd67eb69078cbf8.jpg"
image_tensor = prepare_image(image_path)

image_path1 = "/content/dataset/valid/a427d2734cc7e939_jpg.rf.72b37ad5c1e030b6299ad61ae02da238.jpg"
image_tensor1 = prepare_image(image_path1)

image_path2 = "/content/dataset/valid/9d2d424099458956_jpg.rf.3e56b70f5e4fcc506d9221ee136625ea.jpg"
image_tensor2 = prepare_image(image_path2)

image_path3 = "/content/dataset/valid/f0f3c1c5237b0b1e_jpg.rf.3f6743c8d1005818afa2c089492569e6.jpg"
image_tensor3 = prepare_image(image_path3)

image_path4 = "/content/dataset/valid/fb70621f1a194c73_jpg.rf.e8a16c23912a8a5db5e16c78b70c5491.jpg"
image_tensor4 = prepare_image(image_path4)


with torch.no_grad():
    prediction = model(image_tensor)

draw_boxes(Image.open(image_path), prediction)

with torch.no_grad():
    prediction1 = model(image_tensor1)

draw_boxes(Image.open(image_path1), prediction1)

with torch.no_grad():
    prediction2 = model(image_tensor2)

draw_boxes(Image.open(image_path2), prediction2)

with torch.no_grad():
    prediction3 = model(image_tensor3)

draw_boxes(Image.open(image_path3), prediction3)

with torch.no_grad():
    prediction4 = model(image_tensor4)

draw_boxes(Image.open(image_path4), prediction4)
